In [32]:
from opsml_artifacts import DataCard, CardRegistry, DriftDetector, DriftVisualizer
from opsml_artifacts.connector import SnowflakeQueryRunner
import numpy as np

In [35]:
data_registry = CardRegistry(registry_name="data")

In [36]:
data_registry.list_cards()

,uid,date,timestamp,app_env,name,team,version,user_email,data_uri,drift_uri,feature_map,feature_descriptions,data_splits,data_type,additional_info,dependent_vars
0,b216f555850849d69fd6465b1bd2064e,2023-02-01,1675284706637,staging,test_data,SPMS,2,steven.forrester@shipt.com,gs://shipt-spms-stg-bucket/DATA_REGISTRTY/SPMS...,None,"{'col_0': 'double', 'col_1': 'double', 'col_2'...",None,None,DataFrame,{},[target]
1,e4987cfbbd5b480e88986a3d5be5fff4,2023-02-01,1675285098813,staging,tarp_drop_off,SPMS,2,steven.forrester@shipt.com,gs://shipt-spms-stg-bucket/DATA_REGISTRTY/SPMS...,None,"{'NBR_RX': 'int64', 'APT_FLG': 'int64', 'METRO...",None,"{'splits': [{'label': 'train', 'column': 'EVAL...",DataFrame,{},[DROP_OFF_TIME]
2,2dacc7e21be34902b48a0ee944f18876,2023-02-01,1675284923321,staging,tarp_drop_off,SPMS,1,steven.forrester@shipt.com,gs://shipt-spms-stg-bucket/DATA_REGISTRTY/SPMS...,None,"{'NBR_RX': 'int64', 'APT_FLG': 'int64', 'METRO...",None,"{'splits': [{'label': 'train', 'column': 'EVAL...",DataFrame,{},[DROP_OFF_TIME]
3,93ac43cc817140708028ee3c0ba522ff,2023-02-01,1675285873453,staging,keras_test_data,SPMS,1,steven.forrester@shipt.com,gs://shipt-spms-stg-bucket/DATA_REGISTRTY/SPMS...,None,{'numpy_dtype': 'int32'},None,None,ndarray,{},None
4,d583b63074fa41c3b59097c77e69e6c2,2023-02-01,1675284428556,staging,test_data,SPMS,1,steven.forrester@shipt.com,gs://shipt-spms-stg-bucket/DATA_REGISTRTY/SPMS...,None,"{'col_0': 'double', 'col_1': 'double', 'col_2'...",None,None,DataFrame,{},[target]


In [33]:
query_runner = SnowflakeQueryRunner(on_vpn=True) 
#query runner is a temporary wrapper for pyshipt sql (needed for network issues in vertex, see opsml-pipelines docs)

In [3]:
dataframe = query_runner.query_to_dataframe(sql_file="data.sql") 
#executes sql file or raw sql. data.sql is in examples dir

{"level": "ERROR", "message": "Failed to connect to snowlake. Using API instead. 000606 (57P03): No active warehouse selected in the current session.  Select an active warehouse with the 'use warehouse' command.\n", "timestamp": "2023-02-01T20:58:34.849559Z", "app_env": "development", "host": null, "pid": 338166}


In [34]:
dataframe.head()

,TIME_BUNDLE_ID,LOCAL_CREATED_DATE,LOCAL_CREATED_AT,METRO_ID,BUNDLE_TYPE,NBR_ADDRESSES,NBR_ORDERS,DROP_OFF_TIME,EVAL_FLG,EVAL_OUTLIER,NBR_RX,NBR_APT,DATE_DIFF,TRAIN_OUTLIER,SIN_DAY_WK,COS_DAY_WK,SIN_DAY_YR,COS_DAY_YR,SIN_MTH,COS_MTH,SIN_WK_YR,COS_WK_YR,NG_ORDER_ID,METRO_X,METRO_Y,METRO_Z,APT_FLG
0,28d3147c-f291-4de3-9223-8d337ff33265,2022-12-13,2022-12-13 11:59:48.054941,268,TARP,1,1,1.831357,0,0,0,1,7,0,0.974928,-0.222519,-0.304921,0.952378,-2.449294e-16,1.0,-0.239318,0.970941,E_166350293,-2552.235766,-4607.726400,3583.935130,1
1,34dfa7cd-37bb-402e-a90c-64fc08d8cb61,2022-12-15,2022-12-15 11:26:01.978912,122,TARP,1,1,3.622178,0,0,0,1,7,0,-0.433886,-0.900968,-0.271960,0.962308,-2.449294e-16,1.0,-0.239318,0.970941,E_166719322,1101.043377,-4836.897065,3997.720758,1
2,0fd31e0e-4270-4cf7-b758-e0705cddf318,2022-12-12,2022-12-12 18:39:37.445231,180,TARP,5,5,9.535848,0,0,0,5,7,0,0.781831,0.623491,-0.321271,0.946987,-2.449294e-16,1.0,-0.239318,0.970941,None,267.755064,-5484.710369,3230.464425,1
3,ca67b19a-023c-4793-a8b4-5d33f63fef0f,2022-12-18,2022-12-18 20:47:19.512746,8,TARP,2,2,8.111708,0,0,2,2,7,0,0.000000,1.000000,-0.221919,0.975065,-2.449294e-16,1.0,-0.239318,0.970941,None,971.121016,-5637.844514,2803.796393,1
4,c5bbb865-aaad-42ab-b29d-d9caa355e220,2022-12-17,2022-12-17 13:25:36.317643,48,TARP,1,1,2.703742,0,0,0,1,7,0,-0.781831,0.623491,-0.238672,0.971100,-2.449294e-16,1.0,-0.239318,0.970941,E_167243649,-676.868386,-5314.729422,3447.483337,1


In [37]:
features = [
    "NBR_ADDRESSES",
    "NBR_ORDERS",
    "NBR_RX",
    "NBR_APT",
    "METRO_X",
    "METRO_Y",
    "METRO_Z",
    "APT_FLG",
    "DROP_OFF_TIME",
    "EVAL_FLG",
]
DEPENDENT_VAR = "DROP_OFF_TIME"

# Define DataCard attributes (see examples dir for more detailed information)
DATA_NAME = "tarp_drop_off"
TEAM = "SPMS"
USER_EMAIL = "steven.forrester@shipt.com"
DATA_SPLITS = [
    {"label": "train", "column": "EVAL_FLG", "column_value": 0},
    {"label": "test", "column": "EVAL_FLG", "column_value": 1},
]

data_card = DataCard(
    data=dataframe[features],
    name=DATA_NAME,
    team=TEAM,
    user_email=USER_EMAIL,
    data_splits=DATA_SPLITS,
    dependent_vars=[DEPENDENT_VAR],

)

data_registry = CardRegistry(registry_name="data")
data_registry.register_card(card=data_card)

{"level": "INFO", "message": "DATA_REGISTRTY: tarp_drop_off registered as version 3", "timestamp": "2023-02-01T21:57:29.881147Z", "app_env": "development", "host": null, "pid": 338166}


In [ ]:
DataCard(

In [40]:
from opsml_artifacts import CardRegistry

data_registry = CardRegistry(registry_name="data")
tarp_list = data_registry.list_cards(team="SPMS", name="tarp_drop_off")

In [41]:
print(tarp_list.loc[:, ~tarp_list.columns.isin(["feature_map", "data_splits", "drift_uri"])].to_markdown())

|    | uid                              | date       |     timestamp | app_env   | name          | team   |   version | user_email                 | data_uri                                                                                                        | feature_descriptions   | data_type   | additional_info   | dependent_vars    |
|---:|:---------------------------------|:-----------|--------------:|:----------|:--------------|:-------|----------:|:---------------------------|:----------------------------------------------------------------------------------------------------------------|:-----------------------|:------------|:------------------|:------------------|
|  0 | e1f15bf68efd498cbceb8c540bbd1589 | 2023-02-01 | 1675285098813 | staging   | tarp_drop_off | SPMS   |         3 | steven.forrester@shipt.com | gs://shipt-spms-stg-bucket/DATA_REGISTRTY/SPMS/tarp_drop_off/version-3/e1652db5cb9b4b7ebad687c88321a8d6.parquet |                        | DataFrame   | {}            

In [43]:
loaded_card = data_registry.load_card(uid="d583b63074fa41c3b59097c77e69e6c2") 
loaded_card.load_data() # data is not automatically loaded with the card (prevents loading issues with big data)
# load_card can take a few arguments. Be sure to check to docstring

In [12]:
print(loaded_card.data.head().to_markdown())

|    |   NBR_ADDRESSES |   NBR_ORDERS |   NBR_RX |   NBR_APT |   METRO_X |   METRO_Y |   METRO_Z |   APT_FLG |   DROP_OFF_TIME |   EVAL_FLG |
|---:|----------------:|-------------:|---------:|----------:|----------:|----------:|----------:|----------:|----------------:|-----------:|
|  0 |               1 |            1 |        0 |         1 | -2552.24  |  -4607.73 |   3583.94 |         1 |         1.83136 |          0 |
|  1 |               1 |            1 |        0 |         1 |  1101.04  |  -4836.9  |   3997.72 |         1 |         3.62218 |          0 |
|  2 |               5 |            5 |        0 |         5 |   267.755 |  -5484.71 |   3230.46 |         1 |         9.53585 |          0 |
|  3 |               2 |            2 |        2 |         2 |   971.121 |  -5637.84 |   2803.8  |         1 |         8.11171 |          0 |
|  4 |               1 |            1 |        0 |         1 |  -676.868 |  -5314.73 |   3447.48 |         1 |         2.70374 |          0 |


In [46]:
from opsml_artifacts import ModelCard
from lightgbm import LGBMRegressor

data_splits = data_card.split_data()

In [51]:
data_splits = data_card.split_data() # get the data splits defined by split logic (data_card.data_splits)

# Prepare train data
data_splits.train.pop("EVAL_FLG") # pop off eval flg
y_train = data_splits.train.pop("DROP_OFF_TIME") # get train target

# Prepare test data
data_splits.test.pop("EVAL_FLG") # pop off eval flg
y_test = data_splits.test.pop("DROP_OFF_TIME") # get train target

#### Drift report

In [52]:
drift_detector = DriftDetector(x_reference=data_splits.train,
                              x_current=data_splits.test,
                              y_reference=y_train.to_numpy().reshape(-1,1),
                              y_current=y_test.to_numpy().reshape(-1,1),
                              dependent_var_name="DROP_OFF_TIME",
                             categorical_features=["NBR_ORDERS"],
                             )
drift_detector.visualize_report()

alt.VConcatChart(...)

In [53]:
from opsml_artifacts import ModelCard
from lightgbm import LGBMRegressor

model_registry = CardRegistry(registry_name="model") #load the model registry

data_splits = data_card.split_data() # get the data splits defined by split logic (data_card.data_splits)

# Prepare train data
data_splits.train.pop("EVAL_FLG") # pop off eval flg
y_train = data_splits.train.pop("DROP_OFF_TIME") # get train target

# Prepare test data
data_splits.test.pop("EVAL_FLG") # pop off eval flg
y_test = data_splits.test.pop("DROP_OFF_TIME") # get test target

# fit model
lgb_model = LGBMRegressor()
lgb_model.fit(data_splits.train, y_train)

model_card = ModelCard(
    trained_model=lgb_model,
    sample_input_data=data_splits.train[:1],
    name="tarp_lgb",
    team=TEAM, # defined above
    user_email=USER_EMAIL, # defined above
    data_card_uid=data_card.uid # this is required if you are planning on registering the model
)

{"level": "INFO", "message": "Registering lightgbm onnx converter", "timestamp": "2023-02-01T22:10:03.405084Z", "app_env": "development", "host": null, "pid": 338166}
{"level": "INFO", "message": "Validating converted onnx model", "timestamp": "2023-02-01T22:10:03.681236Z", "app_env": "development", "host": null, "pid": 338166}
{"level": "INFO", "message": "Onnx model validated", "timestamp": "2023-02-01T22:10:03.704430Z", "app_env": "development", "host": null, "pid": 338166}


In [57]:
model_registry = CardRegistry(registry_name="model")
model_registry.register_card(card=model_card)

{"level": "INFO", "message": "MODEL_REGISTRY: tarp_lgb registered as version 2", "timestamp": "2023-02-01T22:11:27.126378Z", "app_env": "development", "host": null, "pid": 338166}


In [60]:
model_card = model_registry.load_card(uid=model_card.uid)
onnx_model = model_card.model()

In [25]:
onnx_model.input_sig.schema()

{'title': 'Features',
 'type': 'object',
 'properties': {'NBR_ADDRESSES': {'title': 'Nbr Addresses', 'type': 'integer'},
  'NBR_ORDERS': {'title': 'Nbr Orders', 'type': 'integer'},
  'NBR_RX': {'title': 'Nbr Rx', 'type': 'integer'},
  'NBR_APT': {'title': 'Nbr Apt', 'type': 'integer'},
  'METRO_X': {'title': 'Metro X', 'type': 'number'},
  'METRO_Y': {'title': 'Metro Y', 'type': 'number'},
  'METRO_Z': {'title': 'Metro Z', 'type': 'number'},
  'APT_FLG': {'title': 'Apt Flg', 'type': 'integer'}},
 'required': ['NBR_ADDRESSES',
  'NBR_ORDERS',
  'NBR_RX',
  'NBR_APT',
  'METRO_X',
  'METRO_Y',
  'METRO_Z',
  'APT_FLG']}

In [63]:
model_card.load_trained_model() # load trained model if laoding from registry

In [64]:
onnx_model.input_sig.schema()

{'title': 'Features',
 'type': 'object',
 'properties': {'NBR_ADDRESSES': {'title': 'Nbr Addresses', 'type': 'integer'},
  'NBR_ORDERS': {'title': 'Nbr Orders', 'type': 'integer'},
  'NBR_RX': {'title': 'Nbr Rx', 'type': 'integer'},
  'NBR_APT': {'title': 'Nbr Apt', 'type': 'integer'},
  'METRO_X': {'title': 'Metro X', 'type': 'number'},
  'METRO_Y': {'title': 'Metro Y', 'type': 'number'},
  'METRO_Z': {'title': 'Metro Z', 'type': 'number'},
  'APT_FLG': {'title': 'Apt Flg', 'type': 'integer'}},
 'required': ['NBR_ADDRESSES',
  'NBR_ORDERS',
  'NBR_RX',
  'NBR_APT',
  'METRO_X',
  'METRO_Y',
  'METRO_Z',
  'APT_FLG']}

In [65]:
record = data_splits.test[0:1].to_dict(orient='records')[0]

# if testing a model that was trained on a numpy array, the model will expect a dictionary with a single list
# record = {"data": list(np.ravel(data[:1]))}

# test the onnx model 
onnx_pred = float(np.ravel(onnx_model.predict(record))[0])

# Compare to original model
orig_pred = float(onnx_model.predict_with_model(model_card.trained_model, record)[0])

print(f"Onnx: {round(onnx_pred,4)}", f"Lightgbm: {round(orig_pred,4)}")

Onnx: 4.908 Lightgbm: 4.908


In [66]:
model_registry.register_card(model_card)

{"level": "INFO", "message": "MODEL_REGISTRY: tarp_lgb registered as version 3", "timestamp": "2023-02-01T22:17:51.910633Z", "app_env": "development", "host": null, "pid": 338166}


In [67]:
model_registry.list_cards()

,uid,date,timestamp,app_env,name,team,version,user_email,model_card_uri,trained_model_uri,sample_data_uri,sample_data_type,model_type
0,da385accf1934374a42c63eeb82d5531,2023-02-01,1675285098813,staging,tarp_lgb,SPMS,3,steven.forrester@shipt.com,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,DataFrame,lgbmregressor
1,328520324fed4a178159290a45a47c15,2023-02-01,1675285098813,staging,tarp_lgb,SPMS,2,steven.forrester@shipt.com,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,DataFrame,lgbmregressor
2,25a2f5882f934c28b0dae89add19b263,2023-02-01,1675285103827,staging,test_random_forest_model,SPMS,1,steven.forrester@shipt.com,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,DataFrame,sklearn_estimator
3,2ffc8de68d4043219cebe2022bfbe510,2023-02-01,1675285105024,staging,test_lin_reg_model,SPMS,1,steven.forrester@shipt.com,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,DataFrame,sklearn_estimator
4,554ef8b60f25427789095000416b325a,2023-02-01,1675285873453,staging,transformer,SPMS,1,steven.forrester@shipt.com,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,ndarray,keras
5,5d52ebc9332942b09eb9d4f1e8524989,2023-02-01,1675285098813,staging,tarp_lgb,SPMS,1,steven.forrester@shipt.com,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,DataFrame,lgbmregressor
6,9eccddb62c4342eca1bac23edc1768d9,2023-02-01,1675284598962,staging,test_lgb_model,SPMS,1,steven.forrester@shipt.com,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,gs://shipt-spms-stg-bucket/MODEL_REGISTRY/SPMS...,DataFrame,lgbmregressor
